In [ ]:
import matplotlib.pyplot as plt

from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_device_benchmarking.bench_code.mrb import MirrorRB

from qiskit.providers.fake_provider import GenericBackendV2

# custom noisy backend (or an attempt at one, at least)
class NoisyBackend(GenericBackendV2):
    def __init__(
        self,
        num_qubits: int,
        basis_gates: list[str] | None = None,
        coupling_map: list[list[int]] = None,
        p1: float = 0,
        p2: float = 0,
    ):
        self.p = (p1,p2)
        super().__init__(
            num_qubits,
            basis_gates,
            coupling_map=coupling_map,
            noise_info = (p1>0 or p2 >0)
            )
    def _get_noise_defaults(self, name: str, num_qubits: int) -> tuple:
        if name in ['delay', 'reset']:
            return (self.p[0],self.p[0])
        else:
            if num_qubits == 1:
                return (0,0,self.p[0],self.p[0])
            else:
                return (0,0,self.p[1],self.p[1])

Pick a backend.

In [ ]:
p = 1e-2
backend = NoisyBackend(
    num_qubits=8,
    basis_gates = ["id", "h", "x", "y", "z", "rx", "cx"],
    p1=p/10,
    p2=p,
)

backend = FakeSherbrooke()

In [ ]:
# number of shots per circuit
shots = 10000

# lengths of different mirror circuits to run
lengths = [2]+[4,10,20,50,100]

# upper bound the qubit number or sherbrooke doesn't work
num_qubits = min(backend.num_qubits,10)

# set up the experiment object
exp = MirrorRB(
    range(num_qubits),
    lengths,
    backend=backend,
    two_qubit_gate_density=0.25,
    num_samples=20,
    )
exp.set_run_options(
    shots=shots
)

Now we run it!

In [ ]:
#run
rb_data = exp.run()
print(rb_data.job_ids)

In [ ]:
exp.analysis.set_options(analyzed_quantity='Effective Polarization')
analysis = exp.analysis.run(rb_data)

In [ ]:
analysis.block_for_results()
analysis.figure(0)